# ML в Биологии
## Random Forest. Part I.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import seaborn as sns
import scipy.stats
from tqdm import tqdm_notebook
from sklearn import datasets
from sklearn.model_selection import GridSearchCV

import warnings
warnings.simplefilter("ignore", DeprecationWarning)

sns.set(context='poster')
%matplotlib inline

### Задача 1:

Скачайте датасет про классификацию женщин с диабетом. Разделите на трейн и тест.

In [ ]:
!unzip archive

In [ ]:
data = pd.read_csv("diabetes.csv")

In [ ]:
data

In [ ]:
X = data[data.columns[:-1]]
y = data[data.columns[-1]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Обучите обычное решающее дерево глубины 3. Выведите его графическое представление, а также accuracy на тренировочной и тестовой выборке.

In [ ]:
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X_train, y_train)
tree.get_params()

Посчитайте accuracy

In [ ]:
print(f'Train accuracy = {accuracy_score(y_train, tree.predict(X_train))}')
print(f'Test accuracy = {accuracy_score(y_test, tree.predict(X_test))}')

Визуализируйте дерево

In [ ]:
plt.figure(figsize=(15, 10))
plot_tree(tree)
plt.show()

Теперь обучите случайный лес с 100 деревьями.

In [ ]:
n_estimators = 100

forest = RandomForestClassifier(n_estimators=n_estimators)

forest.fit(X_train, y_train)

Выведите параметры леса.

In [ ]:
forest.get_params()

Какие параметры уже известны и относятся к решающему дереву?

**Ответ:**

Проще перечислить те, которые уже к лесу относятся: `bootstrap`, `n_estimators`, `n_jobs`, `oob_score`, `verbose`, `warm_start`



Появляются новые важные гиперпараметры: n_estimators (количество деревьев в лесу). Также важным является гиперапараметр max_features &mdash; максимальное количество признаков, которые могут быть перебраны при разбиении вершины дерева. Перед каждым разбиением дерева генерируется выборка из `min(k, max_features)` случайных признаков (`k` — количество признаков в датасете) и только эти признаки рассматриваются как разделяющие в данной вершине.

Выведите качество, сравните с деревом.

In [ ]:
print(f'Train accuracy = {accuracy_score(y_train, forest.predict(X_train))}')
print(f'Test accuracy = {accuracy_score(y_test, forest.predict(X_test))}')

**Вывод:**

 Видно явное переобучение, так как на train лес выучил все, на test стало лучше, но не очень сильно


Нарисуйте первые 6 деревьев леса.

In [ ]:
i = 1
for tree_in_forest in forest.estimators_:
    plt.figure(figsize=(10, 7))
    plot_tree(tree_in_forest)
    plt.title(f'{i}-е дерево леса')
    plt.show()
    i += 1
    if i > 6:
        break

Как выглядят деревья в решающем дереве? Насколько они отличаются от обученного ранее дерева и друг от друга?

**Вывод:**

Каждое дерево в лесу имеет большую глубину и сложную структуру (в отличие от ранее построенного дерева). При этом каждое дерево ветвится по-своему, с разной глубиной, но все они довольно объёмные. Параметры каждой вершины трудно различимы из-за обилия узлов и мелкого текста.